Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License [2017] Zalando SE, https://tech.zalando.com

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/work-with-data/datasets-tutorial/pipeline-with-datasets/pipeline-for-image-classification.png)

# Build a simple ML pipeline for image classification

## Introduction
This tutorial shows how to train a simple deep neural network using the [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) dataset and Keras on Azure Machine Learning. Fashion-MNIST is a dataset of Zalando's article images—consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.

Learn how to:

> * Set up your development environment
> * Create the Fashion MNIST dataset
> * Create a machine learning pipeline to train a simple deep learning neural network on a remote cluster
> * Retrieve input datasets from the experiment and register the output model with datasets

## Prerequisite:
* Understand the [architecture and terms](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture) introduced by Azure Machine Learning
* If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, go through the [configuration notebook](../../../configuration.ipynb) to:
    * install the latest version of AzureML SDK
    * create a workspace and its configuration file (`config.json`)

## Install private build with output dataset feature

In [ ]:
!pip install --extra-index-url https://azuremlsdktestpypi.azureedge.net/Create-Dev-Index/15335858/ --pre "azureml-sdk[automl]<0.1.1"

## Set up your development environment

All the setup for your development work can be accomplished in a Python notebook.  Setup includes:

* Importing Python packages
* Connecting to a workspace to enable communication between your local computer and remote resources
* Creating an experiment to track all your runs
* Creating a remote compute target to use for training

### Import packages

Import Python packages you need in this session. Also display the Azure Machine Learning SDK version.

In [1]:
import os
import azureml.core
from azureml.core import Workspace, Dataset, Datastore, ComputeTarget, RunConfiguration, Experiment
from azureml.core.runconfig import CondaDependencies
from azureml.pipeline.steps import PythonScriptStep, EstimatorStep
from azureml.pipeline.core import Pipeline
# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  0.1.0.15772943


### Connect to workspace

Create a workspace object from the existing workspace. `Workspace.from_config()` reads the file **config.json** and loads the details into an object named `workspace`.

In [2]:
# load workspace
workspace = Workspace.from_config()
print('Workspace name: ' + workspace.name, 
      'Azure region: ' + workspace.location, 
      'Subscription id: ' + workspace.subscription_id, 
      'Resource group: ' + workspace.resource_group, sep='\n')

WARNING - Warning: Falling back to use azure cli login credentials.
If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Workspace name: playground-scus
Azure region: southcentralus
Subscription id: 321cae6f-e4d3-40bf-824f-c07493a62af5
Resource group: aml-testing-scus


### Create experiment and a directory

Create an experiment to track the runs in your workspace and a directory to deliver the necessary code from your computer to the remote resource.

In [3]:
# create an ML experiment
exp = Experiment(workspace=workspace, name='keras-mnist-fashion')

# create a directory
script_folder = './keras-mnist-fashion'
os.makedirs(script_folder, exist_ok=True)

### Create or Attach existing compute resource
By using Azure Machine Learning Compute, a managed service, data scientists can train machine learning models on clusters of Azure virtual machines. Examples include VMs with GPU support. In this tutorial, you create Azure Machine Learning Compute as your training environment. The code below creates the compute clusters for you if they don't already exist in your workspace.

**Creation of compute takes approximately 5 minutes.** If the AmlCompute with that name is already in your workspace the code will skip the creation process.

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "cpu-cluster-1"

try:
    compute_target = ComputeTarget(workspace=workspace, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(workspace, cluster_name, compute_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 1, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-06-16T15:22:50.161000+00:00', 'errors': None, 'creationTime': '2020-06-16T14:53:20.511976+00:00', 'modifiedTime': '2020-06-16T14:53:37.718473+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 1, 'nodeIdleTimeBeforeScaleDown': 'PT7200S'}, 'vmPriority': 'LowPriority', 'vmSize': 'STANDARD_DS3_V2'}


## Create the Fashion MNIST dataset

By creating a dataset, you create a reference to the data source location. If you applied any subsetting transformations to the dataset, they will be stored in the dataset as well. The data remains in its existing location, so no extra storage cost is incurred. 

Every workspace comes with a default [datastore](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-access-data) (and you can register more) which is backed by the Azure blob storage account associated with the workspace. We can use it to transfer data from local to the cloud, and create a dataset from it. We will now upload the [Fashion MNIST](./keras-mnist-fashion) to the default datastore (blob) within your workspace.

In [5]:
# get adls gen 2 datastore already registered with the workspace
datastore = workspace.datastores['may_adlsgen2']

Then we will create an unregistered FileDataset pointing to the path in the datastore. You can also create a dataset from multiple paths. [Learn More](https://aka.ms/azureml/howto/createdatasets) 

**Note:** Before running the cell below, make sure you uploaded the the data (t10k-images-idx3-ubyte, t10k-labels-idx1-ubyte, train-images-idx3-ubyte, train-labels-idx1-ubyte) to the `keras-mnist-fashion` folder in your adlsgen2 datastore. You can do so via storage explorer. 

In [16]:
fashion_ds = Dataset.File.from_files([(datastore, 'keras-mnist-fashion')])

# list the files referenced by fashion_ds
fashion_ds.to_path()

['/t10k-images-idx3-ubyte',
 '/t10k-labels-idx1-ubyte',
 '/train-images-idx3-ubyte',
 '/train-labels-idx1-ubyte']

## Build 2-step ML pipeline

The [Azure Machine Learning Pipeline](https://docs.microsoft.com/en-us/azure/machine-learning/service/concept-ml-pipelines) enables data scientists to create and manage multiple simple and complex workflows concurrently. A typical pipeline would have multiple tasks to prepare data, train, deploy and evaluate models. Individual steps in the pipeline can make use of diverse compute options (for example: CPU for data preparation and GPU for training) and languages. [Learn More](https://github.com/Azure/MachineLearningNotebooks/tree/master/how-to-use-azureml/machine-learning-pipelines)


### Step 1: data preparation

In step one, we will load the image and labels from Fashion MNIST dataset into mnist_train.csv and mnist_test.csv

Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255. Both mnist_train.csv and mnist_test.csv contain 785 columns. The first column consists of the class labels, which represent the article of clothing. The rest of the columns contain the pixel-values of the associated image.

In [17]:
# set up the compute environment to install required packages
conda = CondaDependencies.create(pip_packages=['azureml-sdk<0.1.1'],
                                 pip_indexurl='https://azuremlsdktestpypi.azureedge.net/Create-Dev-Index/15335858/')

conda.set_pip_option('--pre')

run_config = RunConfiguration()
run_config.environment.python.conda_dependencies = conda

Intermediate data (or output of a step) is represented by a `OutputFileDatasetConfig` object. preprared_fashion_ds is produced as the output of step 1, and used as the input of step 2. `OutputFileDatasetConfig` introduces a data dependency between steps, and creates an implicit execution order in the pipeline. You can register a `OutputFileDatasetConfig` as a dataset and version the output data automatically.

In [22]:
from azureml.data import OutputFileDatasetConfig

# learn more about the output config
help(OutputFileDatasetConfig)

Help on class OutputFileDatasetConfig in module azureml.data.output_dataset_config:

class OutputFileDatasetConfig(OutputDatasetConfig)
 |  .. note::This is an experimental class, and may change at any time. For more information, see (https://aka.ms/azuremlexperimental).Represent how to copy the output of a run and be promoted as a FileDataset.
 |  
 |  Method resolution order:
 |      OutputFileDatasetConfig
 |      OutputDatasetConfig
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, name=None, destination=None, source=None)
 |      Initialize a OutputFileDatasetConfig.
 |      
 |      .. remarks::
 |          You can pass the OutputFileDatasetConfig as an argument to your run, and it will be automatically
 |          translated into local path on the compute. The source argument will be used if one is specified,
 |          otherwise we will automatically generate a directory in the OS's temp folder. The files and folders inside
 |          the source d

In [19]:
# write output to adlsgen2 datastore under folder `outputdataset` and registger it as a dataset after the experiment completes
# make sure the service principal in your adlsgen2 datastore has blob data contributor role in order to write data back
prepared_fashion_ds = OutputFileDatasetConfig(destination=(datastore, 'outputdataset/{run-id}')).register_on_complete(name='prepared_fashion_ds')

WARNING - This is an experimental class, and may change at any time. For more information, see (https://aka.ms/azuremlexperimental).
WARNING - This is an experimental class, and may change at any time. For more information, see (https://aka.ms/azuremlexperimental).
WARNING - This is an experimental class, and may change at any time. For more information, see (https://aka.ms/azuremlexperimental).


A **PythonScriptStep** is a basic, built-in step to run a Python Script on a compute target. It takes a script name and optionally other parameters like arguments for the script, compute target, inputs and outputs. If no compute target is specified, default compute target for the workspace is used. You can also use a [**RunConfiguration**](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.runconfiguration?view=azure-ml-py) to specify requirements for the PythonScriptStep, such as conda dependencies and docker image.

In [23]:
# Example is less clear without using argparse and things like arguments = ['--input-dataset', fashion_ds.as_named..., '--output-...']
prep_step = PythonScriptStep(name='prepare step',
                             script_name="prepare.py",
                             # mount fashion_ds dataset to the compute_target
                             arguments=[fashion_ds.as_named_input('fashion_ds').as_mount(), prepared_fashion_ds],
                             source_directory=script_folder,
                             compute_target=compute_target,
                             runconfig=run_config,
                             allow_reuse=True)

### Step 2: train CNN with Keras

Next, we construct an `azureml.train.dnn.TensorFlow` estimator object. The TensorFlow estimator is providing a simple way of launching a TensorFlow training job on a compute target. It will automatically provide a docker image that has TensorFlow installed.

[EstimatorStep](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-steps/azureml.pipeline.steps.estimator_step.estimatorstep?view=azure-ml-py) adds a step to run Tensorflow Estimator in a Pipeline. It takes a dataset as the input.

In [24]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

conda_env = Environment('conda-env')
conda_env.python.conda_dependencies = CondaDependencies.create(pip_packages=['azureml-sdk<0.1.1','keras','tensorflow','numpy','scikit-learn', 'matplotlib','pandas'],
                                                               pip_indexurl='https://azuremlsdktestpypi.azureedge.net/Create-Dev-Index/15335858/')

In [25]:
# No big deal but we're talking above about using the TensorFlow estimator... and we're not using it.
from azureml.train.estimator import Estimator
# set up training step with Estimator
est = Estimator(entry_script='train.py',
                source_directory=script_folder,                 
                environment_definition=conda_env,
                compute_target=compute_target)

# For ease of understanding, and compatiblity with existing code, I would convert the train.py to instead retrive
# the argument using argparse versus Run.get_context().input_datasets['...']
est_step = EstimatorStep(name='train step',
                         estimator=est,
                         estimator_entry_script_arguments=[prepared_fashion_ds.read_delimited_files().as_input(name='prepared_fashion_ds')],
                         compute_target=compute_target)

WARNING - This is an experimental class, and may change at any time. For more information, see (https://aka.ms/azuremlexperimental).
WARNING - This is an experimental class, and may change at any time. For more information, see (https://aka.ms/azuremlexperimental).


### Build the pipeline
Once we have the steps (or steps collection), we can build the [pipeline](https://docs.microsoft.com/python/api/azureml-pipeline-core/azureml.pipeline.core.pipeline.pipeline?view=azure-ml-py).

A pipeline is created with a list of steps and a workspace. Submit a pipeline using [submit](https://docs.microsoft.com/python/api/azureml-core/azureml.core.experiment(class)?view=azure-ml-py#submit-config--tags-none----kwargs-). When submit is called, a [PipelineRun](https://docs.microsoft.com/python/api/azureml-pipeline-core/azureml.pipeline.core.pipelinerun?view=azure-ml-py) is created which in turn creates [StepRun](https://docs.microsoft.com/python/api/azureml-pipeline-core/azureml.pipeline.core.steprun?view=azure-ml-py) objects for each step in the workflow.

In [26]:
# build pipeline & run experiment
pipeline = Pipeline(workspace, steps=[prep_step, est_step])
run = exp.submit(pipeline)

Created step prepare step [93bb3761][5c601cf1-fb39-4341-b4c7-3204499458eb], (This step will run and generate new outputs)
Created step train step [d2d5ae79][985859b0-9482-4523-aa6f-d8ba4a1d2ce2], (This step will run and generate new outputs)
Submitted PipelineRun 6daf7713-76d8-4767-a275-bc27f71ca2f1
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/keras-mnist-fashion/runs/6daf7713-76d8-4767-a275-bc27f71ca2f1?wsid=/subscriptions/321cae6f-e4d3-40bf-824f-c07493a62af5/resourcegroups/aml-testing-scus/workspaces/playground-scus


### Monitor the PipelineRun

In [ ]:
run.wait_for_completion(show_output=True)

PipelineRunId: 6daf7713-76d8-4767-a275-bc27f71ca2f1
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/keras-mnist-fashion/runs/6daf7713-76d8-4767-a275-bc27f71ca2f1?wsid=/subscriptions/321cae6f-e4d3-40bf-824f-c07493a62af5/resourcegroups/aml-testing-scus/workspaces/playground-scus
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 7421c6d6-8cef-4788-8fdb-e49abd914f0e
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/keras-mnist-fashion/runs/7421c6d6-8cef-4788-8fdb-e49abd914f0e?wsid=/subscriptions/321cae6f-e4d3-40bf-824f-c07493a62af5/resourcegroups/aml-testing-scus/workspaces/playground-scus
StepRun( prepare step ) Status: NotStarted

Streaming azureml-logs/20_image_build_log.txt
StepRun( prepare step ) Status: Running
2020/06/16 16:47:46 Downloading source code...
2020/06/16 16:47:48 Finished downloading source code
2020/06/16 16:47:49 Creating Docker network: acb_default_network, driver: 'bridge'
2020/06/16 16:47:49 Succes

In [ ]:
run.find_step_run('train step')[0].get_metrics()

## Register the input dataset and the output model

Azure Machine Learning dataset makes it easy to trace how your data is used in ML. [Learn More](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-version-track-datasets#track-datasets-in-experiments)<br>
For each Machine Learning experiment, you can easily trace the datasets used as the input through `Run` object.

In [ ]:
# get input datasets
prep_step = run.find_step_run('prepare step')[0]
# Why not a method to directly get input datasets? in addition to the get_details()... something like step.get_input_datasets(), or get_output_datasets()
inputs = prep_step.get_details()['inputDatasets']
input_dataset = inputs[0]['dataset']

# list the files referenced by input_dataset
input_dataset.to_path()

Register the input Fashion MNIST dataset with the workspace so that you can reuse it in other experiments or share it with your colleagues who have access to your workspace.

In [ ]:
fashion_ds = input_dataset.register(workspace = workspace,
                                    name = 'fashion_ds',
                                    description = 'image and label files from fashion mnist',
                                    create_new_version = True)
fashion_ds

Register the output model with dataset

In [ ]:
run.find_step_run('train step')[0].register_model(model_name = 'keras-model', model_path = 'outputs/model/', 
                                                  datasets =[('train test data',fashion_ds)])